<a href="https://colab.research.google.com/github/fbeilstein/presentations/blob/master/presentation_for_June_27_2023_lecture_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Modifications of simple gradient descent**

| Optimizer | Year | Learning rate $\alpha$ | Gradient $\nabla f$
|----|---|---|---
Standard  GD|1847 | ||
Momentum|1964| |               $\checkmark$ |
AdaGrad |2011| $\checkmark$ |               |
RMSprop |2011| $\checkmark$ |               |
Adadelta|2012| $\checkmark$ |               |
Nesterov|2013| |               $\checkmark$ |
Adam    |2014| $\checkmark$ | $\checkmark$|
AdaMax  |2015| $\checkmark$ | $\checkmark$|
Nadam   |2015| $\checkmark$ | $\checkmark$|
AMSGrad |2018| $\checkmark$ | $\checkmark$|

Here is a glossary
$$
\begin{aligned}
\text{AdaGrad} &= \text{adaptive gradient} \\
\text{RMSProp} &= \text{root mean square propagation} \\
\text{Adam} &= \text{adaptive movement} \\
\text{Nadam} &= \text{Nesterov + adam} \\
\text{Adadelta} &= \text{adaptive delta} \\
\end{aligned}
$$

In [ ]:
#@title #First order methods and challenging function ("Хто в ліс, хто по дрова...")


from IPython.display import display
import ipywidgets as widgets


class Memory:
  def __init__(self):
    self.x_ini = -2.5
    self.y_ini = -2.5
    self.iters = 30
    self.theta = 0.4

current_memory = Memory()



from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np


import tensorflow as tf
opt_adam = tf.keras.optimizers.Adam(learning_rate=current_memory.theta)
opt_sgd = tf.keras.optimizers.SGD(learning_rate=current_memory.theta)
opt_adadelta = tf.keras.optimizers.Adadelta(learning_rate=current_memory.theta)
opt_adagrad = tf.keras.optimizers.Adagrad(learning_rate=current_memory.theta)
opt_adamax = tf.keras.optimizers.Adamax(learning_rate=current_memory.theta)
opt_ftrl = tf.keras.optimizers.Ftrl(learning_rate=current_memory.theta)
opt_nadam = tf.keras.optimizers.Nadam(learning_rate=current_memory.theta)
opt_rmsprop = tf.keras.optimizers.RMSprop(learning_rate=current_memory.theta)



x_min = -3.0
x_max = 3.0
y_min = -3.0
y_max = 3.0

scatters = []

for opt, color, name in [[opt_adam, 'red', 'Adam'],
                         [opt_sgd, 'green', 'SGD'],
                         [opt_adadelta, 'blue', 'Adadelta'],
                         [opt_adagrad, 'yellow', 'Adagrad'],
                         [opt_adamax, 'orange', 'Adamax'],
                         [opt_ftrl, 'violet', 'FTRL'],
                         [opt_nadam, 'navy', 'Nadam'],
                         [opt_rmsprop, 'maroon', 'RMSprop']]:
  history = [np.array([current_memory.x_ini, current_memory.y_ini])]
  varx = tf.Variable(current_memory.x_ini)
  vary = tf.Variable(current_memory.y_ini)
  func_to_minimize = lambda: (varx**2 + vary**2) / 2.0 + tf.sin(4*varx)**2 #varx**2 + vary**2
  ftm = lambda x: (x[0]**2 + x[1]**2) / 2.0 + np.sin(4.0 * x[0])**2 #x[0]**2 + x[1]**2

  #O_basins_and_walls = (W**2).sum()/2 + T.sin(W[0][0]*4)**2

  for i in range(current_memory.iters):
    opt.minimize(func_to_minimize, [varx, vary])
    history.append([varx.numpy(), vary.numpy()])

  _h = np.array(history)
  _x = _h.T[0]
  _y = _h.T[1]
  _z = np.array([ftm(x) for x in history], dtype=float)

  x_min = min(x_min, np.min(_x))
  x_max = max(x_max, np.max(_x))
  y_min = min(y_min, np.min(_y))
  y_max = max(y_max, np.max(_y))

  scatters.append({'type': 'scatter',
                   'name': name, 'x': _x, 'y': _y,
                   'line': {'color': color, 'width': 2} })
  scatters.append({'type': 'scatter3d',
                  'name': name + ' (3D)', 'x': _x, 'y': _y, 'z': _z,
                  'line': {'color': color, 'width': 2}, 'marker': {'size': 4} })


def shorten(scatters, k):
  import copy
  new_scatters = copy.deepcopy(scatters)
  for s in new_scatters:
    s['x'] = s['x'][:k]
    s['y'] = s['y'][:k]
  return new_scatters

x_ = np.linspace(x_min, x_max, num=50)
y_ = np.linspace(y_min, y_max, num=50)
z_ = np.array([[ftm([x,y]) for x in x_] for y in y_], dtype=float)

import matplotlib.pyplot as plt
import base64
import io
fig = plt.figure(figsize=(20,20))
ax = fig.gca()
ax.axis('off')
ax.contourf(x_, y_, z_, alpha=0.3)
plt.close(fig)
buf = io.BytesIO()
fig.savefig(buf, format='png', bbox_inches='tight', pad_inches=0);
image_base64 = u'data:image/  png;base64,' + base64.b64encode(buf.getvalue()).decode('utf-8').replace('\n', '')
buf.close()


fig = dict(
    layout = dict(
        width=1200, height=600, autosize=False,
        showlegend = True,
        scene = {'domain': { 'x': [0.0, 0.44], 'y': [0, 1] } },
        xaxis1 = {'domain': [0.55, 1], 'range': [x_min, x_max], 'fixedrange': True},
        yaxis1 = {'domain': [0.0, 1.0], 'range': [y_min, y_max], 'fixedrange': True},
        title  = 'Minimization',
        margin = {'t': 50, 'b': 50, 'l': 50, 'r': 50},
        updatemenus = [{'buttons': [{'args': [[k for k in range(len(_h)-1)],
                                              {'frame': {'duration': 500.0, 'redraw': True},
                                              'fromcurrent': False, 'transition': {'duration': 0, 'easing': 'linear'}}],
                                    'label': 'Play',
                                    'method': 'animate'},
                                    {'args': [[None], {'frame': {'duration': 0, 'redraw': True},
                                                      'mode': 'immediate',
                                                      'transition': {'duration': 0}}
                                              ],
                                    'label': 'Pause',
                                    'method': 'animate'
                                    }
                                    ],
                        'direction': 'left',
                        'pad': {'r': 10, 't': 85},
                        'showactive': True,
                        'type': 'buttons', 'x': 0.1, 'y': 0, 'xanchor': 'right', 'yanchor': 'top'}],
        sliders = [{'yanchor': 'top',
                    'xanchor': 'left',
                    'currentvalue': {'font': {'size': 16}, 'prefix': 'Step: ', 'visible': True, 'xanchor': 'right'},
                    'transition': {'duration': 0.0},
                    'pad': {'b': 10, 't': 50},
                    'len': 0.9,
                    'x': 0.1,
                    'y': 0,
                    'steps': [{'args': [[k], {'frame': {'duration': 500.0, 'easing': 'linear', 'redraw': True},
                                              'transition': {'duration': 0, 'easing': 'linear'}}
                                        ],
                              'label': k,
                              'method': 'animate'} for k in range(len(_h)-1)
                    ]}],
        images = [{'source' : image_base64,
                  'xref': 'x', 'yref': 'y',
                  'sizing': 'stretch',
                  'sizex': x_max - x_min, 'sizey': y_max - y_min,
                  'layer': 'below', 'opacity':1.0,
                  'x': x_min, 'y': y_max}]
    ),
    data = [
        #{'type': 'scatter3d', 'name': 's3', 'x': _x, 'y': _y, 'z': _z, 'line': {'color': 'red', 'width': 2}, 'marker': {'size': 4, 'colorscale': 'Viridis'}},
        {'type': 'surface', 'name': 'f2', 'x': x_, 'y': y_, 'z': z_, 'opacity': 0.8, 'showscale': False},
        #{'type': 'contour', 'name': 'c1', 'x':x_, 'y':y_, 'z':z_, 'contours': {'showlabels': True}},
    ] + scatters,
    frames=[
        {'name': str(k),
        'data': [
          #{'type': 'scatter3d', 'name': 's3', 'x': _x[:k], 'y': _y[:k], 'z': _z[:k], 'line': {'color': 'red', 'width': 2}, 'marker': {'size': 4, 'colorscale': 'Viridis'}},
          {'type': 'surface', 'name': 'f2', 'x': x_, 'y': y_, 'z': z_, 'opacity': 0.8, 'showscale': False},
          #{'type': 'contour', 'name': 'c1', 'x':x_, 'y':y_, 'z':z_, 'contours': {'showlabels': True}},
        ] + shorten(scatters, k+1) } for k in range(len(_h)-1) ]
)
f = go.Figure(fig)
f.show()

In [ ]:
#@title ##Back propagation (how to get gradient)
%%html

<style>
  .MathJax {
font-size: 0.6em;
}
</style>
<script src='https://www.gstatic.com/external_hosted/mathjax/latest/MathJax.js?config=default'>
</script>

<div id="slide_1" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_1.jpg" width="750px">
</div>

<div id="slide_2" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_2.jpg" width="750px">
</div>

<div id="slide_3" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_3.jpg" width="750px">
</div>

<div id="slide_4" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_4.jpg" width="750px">
</div>

<div id="slide_5" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_5.jpg" width="750px">
</div>

<div id="slide_6" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_6.jpg" width="750px">
</div>

<div id="slide_7" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_7.jpg" width="750px">
</div>

<div id="slide_8" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_8.jpg" width="750px">
</div>

<div id="slide_9" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_9.jpg" width="750px">
</div>

<div id="slide_10" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_10.jpg" width="750px">
</div>

<div id="slide_11" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_11.jpg" width="750px">
</div>

<div id="slide_12" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_12.jpg" width="750px">
</div>

<div id="slide_13" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_13.jpg" width="750px">
</div>

<div id="slide_14" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_14.jpg" width="750px">
</div>

<div id="slide_15" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_15.jpg" width="750px">
</div>

<div id="slide_16" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_16.jpg" width="750px">
</div>

<div id="slide_17" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_17.jpg" width="750px">
</div>

<div id="slide_18" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_18.jpg" width="750px">
</div>

<div id="slide_19" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_19.jpg" width="750px">
</div>

<div id="slide_20" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/back_prop/slide_20.jpg" width="750px">
</div>


<div style="position:absolute; top:100px; left:800px;">

<div id="arrow_left" style="border-width: 1px; border-style:solid; float:left; height:32px;">
<svg width="8.7464mm" height="8.7464mm" version="1.1" viewBox="0 0 8.7464 8.7464" xmlns="http://www.w3.org/2000/svg" xmlns:cc="http://creativecommons.org/ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
<metadata>
<rdf:RDF>
<cc:Work rdf:about="">
<dc:format>image/svg+xml</dc:format>
<dc:type rdf:resource="http://purl.org/dc/dcmitype/StillImage"/>
<dc:title/>
</cc:Work>
</rdf:RDF>
</metadata>
<g transform="translate(-38.564 -29.908)">
<rect x="38.564" y="29.908" width="8.7464" height="8.7464" fill-opacity=".16425" opacity=".97" stroke-miterlimit="10.433" stroke-width="0"/>
<path d="m45.796 34.331h-5.8931" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m40.068 34.267 2.7141-2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m40.069 34.396 2.7141 2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
</g>
</svg>
</div>

<div id="arrow_right" style="border-width: 1px; border-style:solid; float:right; height:32px;">
<svg width="8.7464mm" height="8.7464mm" version="1.1" viewBox="0 0 8.7464 8.7464" xmlns="http://www.w3.org/2000/svg" xmlns:cc="http://creativecommons.org/ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
<metadata>
<rdf:RDF>
<cc:Work rdf:about="">
<dc:format>image/svg+xml</dc:format>
<dc:type rdf:resource="http://purl.org/dc/dcmitype/StillImage"/>
<dc:title/>
</cc:Work>
</rdf:RDF>
</metadata>
<g transform="translate(-38.564 -29.908)">
<rect transform="scale(-1,1)" x="-47.31" y="29.908" width="8.7464" height="8.7464" fill-opacity=".16425" opacity=".97" stroke-miterlimit="10.433" stroke-width="0"/>
<path d="m40.078 34.331h5.8931" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m45.806 34.267-2.7141-2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m45.805 34.396-2.7141 2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
</g>
</svg>
</div>

</div>

<style>
#arrow_left:hover{color:blue;}
#arrow_right:hover{color:blue;}
</style>

<script  type="text/javascript">
var slides = ["slide_1", "slide_2", "slide_3", "slide_4",
               "slide_5", "slide_6",
              "slide_7", "slide_8", "slide_9",
              "slide_10", "slide_11", "slide_12",
              "slide_13", "slide_14", "slide_15",
              "slide_16", "slide_17", "slide_18",
              "slide_19", "slide_20"];
var left_btn_id = "arrow_left";
var right_btn_id = "arrow_right";
var current_slide = 0;
var merging = false;

function slide_left()
{
  var old_slide = document.getElementById(slides[current_slide]);
  current_slide -= 1;
  if (current_slide < 0) {current_slide = 0;}
  var new_slide = document.getElementById(slides[current_slide]);
  old_slide.style.display='none';
  new_slide.style.display='block';
  if (current_slide != 5)
      merging = false;
}

function slide_right()
{
  var old_slide = document.getElementById(slides[current_slide]);
  current_slide += 1;
  if (current_slide >= slides.length) {current_slide = slides.length - 1;}
  var new_slide = document.getElementById(slides[current_slide]);
  old_slide.style.display='none';
  new_slide.style.display='block';
  if (current_slide == 5)
  {
    if (!merging)
      merge_images(200);
    merging = true;
  } else {
    merging = false;
  }
}

function image_opacity(image, opacity)
{
    var s = image.style;
    s.opacity = (opacity / 100);
    s.MozOpacity = (opacity / 100);
    s.KhtmlOpacity = (opacity / 100);
    s.filter = 'alpha(opacity=' + opacity + ')';
}

function merge_images(cycles_left)
{
  document.getElementById("cat").style.left = (200 - cycles_left) + "px";
  document.getElementById("pic").style.left = (400 - (200 - cycles_left)) + "px";
  image_opacity(document.getElementById("cat"), cycles_left / 2)
  image_opacity(document.getElementById("pic"), cycles_left / 2)
  image_opacity(document.getElementById("merge"), 100 - cycles_left / 2)
  if (cycles_left > 0)
    setTimeout(function(){merge_images(cycles_left - 1);}, 3);
}

document.getElementById(left_btn_id).addEventListener("click", slide_left);
document.getElementById(right_btn_id).addEventListener("click", slide_right);
document.getElementById(slides[current_slide]).style.display='block';
</script>

In [ ]:
#@title ##NN example
%%html

<style>
  .MathJax {
font-size: 0.6em;
}
</style>
<script src='https://www.gstatic.com/external_hosted/mathjax/latest/MathJax.js?config=default'>
</script>

<div id="slide_1" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_1.jpg" width="750px">
</div>

<div id="slide_2" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_2.jpg" width="750px">
</div>

<div id="slide_3" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_3.jpg" width="750px">
</div>

<div id="slide_4" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_4.jpg" width="750px">
</div>

<div id="slide_5" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_5.jpg" width="750px">
</div>

<div id="slide_6" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_6.jpg" width="750px">
</div>

<div id="slide_7" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_7.jpg" width="750px">
</div>

<div id="slide_8" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_8.jpg" width="750px">
</div>

<div id="slide_9" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_9.jpg" width="750px">
</div>

<div id="slide_10" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_10.jpg" width="750px">
</div>

<div id="slide_11" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_11.jpg" width="750px">
</div>

<div id="slide_12" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_12.jpg" width="750px">
</div>

<div id="slide_13" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_13.jpg" width="750px">
</div>

<div id="slide_14" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_14.jpg" width="750px">
</div>

<div id="slide_15" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_15.jpg" width="750px">
</div>

<div id="slide_16" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_16.jpg" width="750px">
</div>

<div id="slide_17" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_17.jpg" width="750px">
</div>

<div id="slide_18" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_18.jpg" width="750px">
</div>

<div id="slide_19" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_19.jpg" width="750px">
</div>

<div id="slide_20" style="display:none; max-width:750px">
  <img src="https://raw.githubusercontent.com/fbeilstein/presentations/master/images/nn_example/slide_20.jpg" width="750px">
</div>


<div style="position:absolute; top:100px; left:800px;">

<div id="arrow_left" style="border-width: 1px; border-style:solid; float:left; height:32px;">
<svg width="8.7464mm" height="8.7464mm" version="1.1" viewBox="0 0 8.7464 8.7464" xmlns="http://www.w3.org/2000/svg" xmlns:cc="http://creativecommons.org/ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
<metadata>
<rdf:RDF>
<cc:Work rdf:about="">
<dc:format>image/svg+xml</dc:format>
<dc:type rdf:resource="http://purl.org/dc/dcmitype/StillImage"/>
<dc:title/>
</cc:Work>
</rdf:RDF>
</metadata>
<g transform="translate(-38.564 -29.908)">
<rect x="38.564" y="29.908" width="8.7464" height="8.7464" fill-opacity=".16425" opacity=".97" stroke-miterlimit="10.433" stroke-width="0"/>
<path d="m45.796 34.331h-5.8931" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m40.068 34.267 2.7141-2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m40.069 34.396 2.7141 2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
</g>
</svg>
</div>

<div id="arrow_right" style="border-width: 1px; border-style:solid; float:right; height:32px;">
<svg width="8.7464mm" height="8.7464mm" version="1.1" viewBox="0 0 8.7464 8.7464" xmlns="http://www.w3.org/2000/svg" xmlns:cc="http://creativecommons.org/ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
<metadata>
<rdf:RDF>
<cc:Work rdf:about="">
<dc:format>image/svg+xml</dc:format>
<dc:type rdf:resource="http://purl.org/dc/dcmitype/StillImage"/>
<dc:title/>
</cc:Work>
</rdf:RDF>
</metadata>
<g transform="translate(-38.564 -29.908)">
<rect transform="scale(-1,1)" x="-47.31" y="29.908" width="8.7464" height="8.7464" fill-opacity=".16425" opacity=".97" stroke-miterlimit="10.433" stroke-width="0"/>
<path d="m40.078 34.331h5.8931" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m45.806 34.267-2.7141-2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m45.805 34.396-2.7141 2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
</g>
</svg>
</div>

</div>

<style>
#arrow_left:hover{color:blue;}
#arrow_right:hover{color:blue;}
</style>

<script  type="text/javascript">
var slides = ["slide_1", "slide_2", "slide_3", "slide_4",
               "slide_5", "slide_6",
              "slide_7", "slide_8", "slide_9",
              "slide_10", "slide_11", "slide_12",
              "slide_13", "slide_14", "slide_15",
              "slide_16", "slide_17", "slide_18",
              "slide_19", "slide_20"];
var left_btn_id = "arrow_left";
var right_btn_id = "arrow_right";
var current_slide = 0;
var merging = false;

function slide_left()
{
  var old_slide = document.getElementById(slides[current_slide]);
  current_slide -= 1;
  if (current_slide < 0) {current_slide = 0;}
  var new_slide = document.getElementById(slides[current_slide]);
  old_slide.style.display='none';
  new_slide.style.display='block';
  if (current_slide != 5)
      merging = false;
}

function slide_right()
{
  var old_slide = document.getElementById(slides[current_slide]);
  current_slide += 1;
  if (current_slide >= slides.length) {current_slide = slides.length - 1;}
  var new_slide = document.getElementById(slides[current_slide]);
  old_slide.style.display='none';
  new_slide.style.display='block';
  if (current_slide == 5)
  {
    if (!merging)
      merge_images(200);
    merging = true;
  } else {
    merging = false;
  }
}

function image_opacity(image, opacity)
{
    var s = image.style;
    s.opacity = (opacity / 100);
    s.MozOpacity = (opacity / 100);
    s.KhtmlOpacity = (opacity / 100);
    s.filter = 'alpha(opacity=' + opacity + ')';
}

function merge_images(cycles_left)
{
  document.getElementById("cat").style.left = (200 - cycles_left) + "px";
  document.getElementById("pic").style.left = (400 - (200 - cycles_left)) + "px";
  image_opacity(document.getElementById("cat"), cycles_left / 2)
  image_opacity(document.getElementById("pic"), cycles_left / 2)
  image_opacity(document.getElementById("merge"), 100 - cycles_left / 2)
  if (cycles_left > 0)
    setTimeout(function(){merge_images(cycles_left - 1);}, 3);
}

document.getElementById(left_btn_id).addEventListener("click", slide_left);
document.getElementById(right_btn_id).addEventListener("click", slide_right);
document.getElementById(slides[current_slide]).style.display='block';
</script>

In [ ]:
#@title ##GAN style transfer
%%html


<div id="slide_6" style="display:none; max-width:750px; height:200px;">
  <img src="https://raw.githubusercontent.com/fbeilstein/machine_learning/master/lecture_01_introduction/Cat.jpg"
  style="position:absolute; left:0px;  top:0px;">
  <img src="https://raw.githubusercontent.com/fbeilstein/machine_learning/master/lecture_01_introduction/Candinsky.jpg"
  style="position:absolute; left:400px; top:0px;">
</div>

<div id="slide_7" style="display:none; max-width:750px; height:200px;">
  <img src="https://raw.githubusercontent.com/fbeilstein/machine_learning/master/lecture_01_introduction/Cat.jpg"
  style="position:absolute; left:0px;  top:0px;" id="cat">
  <img src="https://raw.githubusercontent.com/fbeilstein/machine_learning/master/lecture_01_introduction/CandinskyCat.jpg"
  style="position:absolute; left:200px; top:0px;" id="merge">
  <img src="https://raw.githubusercontent.com/fbeilstein/machine_learning/master/lecture_01_introduction/Candinsky.jpg"
  style="position:absolute; left:400px; top:0px;" id="pic">
</div>



<div style="position:absolute; top:100px; left:800px;">

<div id="arrow_left" style="border-width: 1px; border-style:solid; float:left; height:32px;">
<svg width="8.7464mm" height="8.7464mm" version="1.1" viewBox="0 0 8.7464 8.7464" xmlns="http://www.w3.org/2000/svg" xmlns:cc="http://creativecommons.org/ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
<metadata>
<rdf:RDF>
<cc:Work rdf:about="">
<dc:format>image/svg+xml</dc:format>
<dc:type rdf:resource="http://purl.org/dc/dcmitype/StillImage"/>
<dc:title/>
</cc:Work>
</rdf:RDF>
</metadata>
<g transform="translate(-38.564 -29.908)">
<rect x="38.564" y="29.908" width="8.7464" height="8.7464" fill-opacity=".16425" opacity=".97" stroke-miterlimit="10.433" stroke-width="0"/>
<path d="m45.796 34.331h-5.8931" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m40.068 34.267 2.7141-2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m40.069 34.396 2.7141 2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
</g>
</svg>
</div>

<div id="arrow_right" style="border-width: 1px; border-style:solid; float:right; height:32px;">
<svg width="8.7464mm" height="8.7464mm" version="1.1" viewBox="0 0 8.7464 8.7464" xmlns="http://www.w3.org/2000/svg" xmlns:cc="http://creativecommons.org/ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
<metadata>
<rdf:RDF>
<cc:Work rdf:about="">
<dc:format>image/svg+xml</dc:format>
<dc:type rdf:resource="http://purl.org/dc/dcmitype/StillImage"/>
<dc:title/>
</cc:Work>
</rdf:RDF>
</metadata>
<g transform="translate(-38.564 -29.908)">
<rect transform="scale(-1,1)" x="-47.31" y="29.908" width="8.7464" height="8.7464" fill-opacity=".16425" opacity=".97" stroke-miterlimit="10.433" stroke-width="0"/>
<path d="m40.078 34.331h5.8931" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m45.806 34.267-2.7141-2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m45.805 34.396-2.7141 2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
</g>
</svg>
</div>

</div>

<style>
#arrow_left:hover{color:blue;}
#arrow_right:hover{color:blue;}
</style>

<script  type="text/javascript">
var slides = ["slide_6", "slide_7"];
var left_btn_id = "arrow_left";
var right_btn_id = "arrow_right";
var current_slide = 0;
var merging = false;

function slide_left()
{
  var old_slide = document.getElementById(slides[current_slide]);
  current_slide -= 1;
  if (current_slide < 0) {current_slide = 0;}
  var new_slide = document.getElementById(slides[current_slide]);
  old_slide.style.display='none';
  new_slide.style.display='block';
  if (current_slide != 5)
      merging = false;
}

function slide_right()
{
  var old_slide = document.getElementById(slides[current_slide]);
  current_slide += 1;
  if (current_slide >= slides.length) {current_slide = slides.length - 1;}
  var new_slide = document.getElementById(slides[current_slide]);
  old_slide.style.display='none';
  new_slide.style.display='block';
  if (current_slide == 1)
  {
    if (!merging)
      merge_images(200);
    merging = true;
  } else {
    merging = false;
  }
}

function image_opacity(image, opacity)
{
    var s = image.style;
    s.opacity = (opacity / 100);
    s.MozOpacity = (opacity / 100);
    s.KhtmlOpacity = (opacity / 100);
    s.filter = 'alpha(opacity=' + opacity + ')';
}

function merge_images(cycles_left)
{
  document.getElementById("cat").style.left = (200 - cycles_left) + "px";
  document.getElementById("pic").style.left = (400 - (200 - cycles_left)) + "px";
  image_opacity(document.getElementById("cat"), cycles_left / 2)
  image_opacity(document.getElementById("pic"), cycles_left / 2)
  image_opacity(document.getElementById("merge"), 100 - cycles_left / 2)
  if (cycles_left > 0)
    setTimeout(function(){merge_images(cycles_left - 1);}, 3);
}

document.getElementById(left_btn_id).addEventListener("click", slide_left);
document.getElementById(right_btn_id).addEventListener("click", slide_right);
document.getElementById(slides[current_slide]).style.display='block';
</script>